In [13]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [14]:
df = pd.read_excel('All_Cities.xlsx')
df.head()

,year,month,day,SO2,NO2,PM10,Stn Code,State,City
0,2015,1,1,9.0,18.0,46.0,233,Andhra Pradesh,Visakhapatnam
1,2015,1,4,8.0,18.0,27.0,233,Andhra Pradesh,Visakhapatnam
2,2015,1,7,9.0,15.0,29.0,233,Andhra Pradesh,Visakhapatnam
3,2015,1,10,7.0,17.0,78.0,233,Andhra Pradesh,Visakhapatnam
4,2015,1,13,9.0,23.0,35.0,233,Andhra Pradesh,Visakhapatnam


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213083 entries, 0 to 213082
Data columns (total 9 columns):
year        213083 non-null int64
month       213083 non-null int64
day         213083 non-null int64
SO2         213083 non-null float64
NO2         213083 non-null float64
PM10        213083 non-null float64
Stn Code    213083 non-null int64
State       213083 non-null object
City        213083 non-null object
dtypes: float64(3), int64(4), object(2)
memory usage: 14.6+ MB


In [16]:
df.describe()

,year,month,day,SO2,NO2,PM10,Stn Code
count,213083.000000,213083.000000,213083.000000,213083.000000,213083.000000,213083.000000,213083.000000
mean,2013.161787,6.493831,15.642848,9.371937,23.931919,105.794332,434.213133
std,1.413248,3.447950,8.733599,11.053701,18.441429,72.050260,213.693708
min,2011.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,2012.000000,3.000000,8.000000,4.000000,13.000000,55.000000,287.000000
50%,2013.000000,6.000000,16.000000,7.000000,20.000000,88.000000,450.000000
75%,2014.000000,10.000000,23.000000,12.000000,30.000000,138.000000,609.000000
max,2015.000000,12.000000,31.000000,909.000000,876.000000,1288.000000,992.000000


In [17]:
df['SO2'].max()

909.0

In [18]:
df['NO2'].max()

876.0

In [19]:
df['PM10'].max()

1288.0

In [27]:
# lets rescale the measures to 0-500 in order to calculate the AIR QUALITY INDEX

# Converting SO2 measure to SI(0,500)
def calculate_si(so2):
    si=0
    if (so2<=40):
     si= so2*(50/40)
    elif (so2>40 and so2<=80):
     si= 50+(so2-40)*(50/40)
    elif (so2>80 and so2<=380):
     si= 100+(so2-80)*(100/300)
    elif (so2>380 and so2<=800):
     si= 200+(so2-380)*(100/800)
    elif (so2>800 and so2<=1600):
     si= 300+(so2-800)*(100/800)
    else:
     si= 400+(so2-1600)*(100/800)
    return si

# Converting NO2 measures to NI(0,500)
def calculate_ni(no2):
    ni=0
    if(no2<=40):
     ni= no2*50/40
    elif(no2>40 and no2<=80):
     ni= 50+(no2-14)*(50/40)
    elif(no2>80 and no2<=180):
     ni= 100+(no2-80)*(100/100)
    elif(no2>180 and no2<=280):
     ni= 200+(no2-180)*(100/100)
    elif(no2>280 and no2<=400):
     ni= 300+(no2-280)*(100/120)
    else:
     ni= 400+(no2-400)*(100/120)
    return ni

# Converting PM10 to PI(0,500)
def calculate_pi(pm10):
    pi=0
    if(pm10<=50):
     pi=pm10
    elif(pm10>50 and pm10<=100):
     pi=pm10
    elif(pm10>100 and pm10<=250):
     pi=100+(pm10-100)*100/150
    elif(pm10>250 and pm10<=350):
     pi=200+(pm10-250)
    elif(pm10>350 and pm10<=430):
     pi=300+(pm10-350)*(100/80)
    else:
     pi=400+(pm10-430)*(100/80)
    return pi

In [28]:
# Calculating AQI
def calculate_aqi(si,ni,pi):
    aqi=0
    if(si>ni and si>pi):
     aqi=si
    if(pi>si and pi>ni):
     aqi=pi
    if(ni>si and ni>pi):
     aqi=ni
    return aqi

In [29]:
df['si']=df['SO2'].apply(calculate_si)
df['ni']=df['NO2'].apply(calculate_ni)
df['pi']=df['PM10'].apply(calculate_pi)
df['AQI']=df.apply(lambda x:calculate_aqi(x['si'],x['ni'],x['pi']),axis=1)
df.head()

,year,month,day,SO2,NO2,PM10,Stn Code,State,City,si,ni,pi,AQI
0,2015,1,1,9.0,18.0,46.0,233,Andhra Pradesh,Visakhapatnam,11.25,22.50,46.0,46.0
1,2015,1,4,8.0,18.0,27.0,233,Andhra Pradesh,Visakhapatnam,10.00,22.50,27.0,27.0
2,2015,1,7,9.0,15.0,29.0,233,Andhra Pradesh,Visakhapatnam,11.25,18.75,29.0,29.0
3,2015,1,10,7.0,17.0,78.0,233,Andhra Pradesh,Visakhapatnam,8.75,21.25,78.0,78.0
4,2015,1,13,9.0,23.0,35.0,233,Andhra Pradesh,Visakhapatnam,11.25,28.75,35.0,35.0


In [32]:
df['State'].unique()

array(['Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Chandigarh',
       'Chhattisgarh', 'Dadra and Nagar Haveli', 'Daman and Diu', 'Delhi',
       'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh', 'Jammu & Kashmir',
       'Jharkhand', 'Karnataka', 'Kerala', 'Madhya Pradesh',
       'Maharashtra', 'Meghalaya', 'Mizoram', 'Nagaland', 'Odisha',
       'Puducherry', 'Punjab', 'Rajasthan', 'Tamil Nadu', 'Telangana',
       'Uttarakhand', 'Uttar Pradesh', 'West Bengal', 'Bihar'],
      dtype=object)

In [35]:
df.groupby(['State', 'City']).count()

year  month   day   SO2   NO2  PM10  \
State             City                                                  
Andhra Pradesh    Ananthapur       324    324   324   324   324   324   
                  Chittoor         910    910   910   910   910   910   
                  Eluru            300    300   300   300   300   300   
                  Guntur           487    487   487   487   487   487   
                  Hyderabad       2109   2109  2109  2109  2109  2109   
                  Kadapa           316    316   316   316   316   316   
                  Kakinada         288    288   288   288   288   288   
                  Karimnagar        67     67    67    67    67    67   
                  Khammam          282    282   282   282   282   282   
                  Kurnool          432    432   432   432   432   432   
                  Nalgonda         476    476   476   476   476   476   
                  Nellore          313    313   313   313   313   313   
                  Nizamabad         27     27    27    27    27    27   
                  Ongole           317    317   317   317   317   317   
                  Patancheru       321    321   321   321   321   321   
                  Rajahmundry      311    311   311   311   311   311   
                  Ramagundam       200    200   200   200   200   200   
                  Sangareddy        85     85    85    85    85    85   
                  Srikakulam       315    315   315   315   315   315   
                  Tirupati         468    468   468   468   468   468   
                  Vijayawada      1030   1030  1030  1030  1030  1030   
                  Visakhapatnam   3189   3189  3189  3189  3189  3189   
                  Vizianagaram     282    282   282   282   282   282   
                  Warangal         214    214   214   214   214   214   
Arunachal Pradesh Itanagar          45     45    45    45    45    45   
                  Naharlagun        45     45    45    45    45    45   
Assam             Bongaigaon       805    805   805   805   805   805   
                  Daranga          307    307   307   307   307   307   
                  Dibrugarh        378    378   378   378   378   378   
                  Golaghat         358    358   358   358   358   358   
...                                ...    ...   ...   ...   ...   ...   
Uttar Pradesh     Mathura           78     78    78    78    78    78   
                  Meerut           266    266   266   266   266   266   
                  Moradabad        956    956   956   956   956   956   
                  Noida            528    528   528   528   528   528   
                  Rai Bareilly     970    970   970   970   970   970   
                  Renusagar        465    465   465   465   465   465   
                  Saharanpur        46     46    46    46    46    46   
                  Unnao            388    388   388   388   388   388   
                  Varanasi         666    666   666   666   666   666   
Uttarakhand       Dehradun         601    601   601   601   601   601   
                  Haldwani         254    254   254   254   254   254   
                  Haridwar         299    299   299   299   299   299   
                  Kashipur         236    236   236   236   236   236   
                  Rishikesh        215    215   215   215   215   215   
                  Rudrapur         213    213   213   213   213   213   
West Bengal       Asansol          704    704   704   704   704   704   
                  Barrackpore      544    544   544   544   544   544   
                  DANKUNI          103    103   103   103   103   103   
                  Durgapur         325    325   325   325   325   325   
                  HALDIA           104    104   104   104   104   104   
                  Haldia           650    650   650   650   650   650   
                  Howrah           649    649   649   649   649   649   
                  K

In [40]:
df.to_excel('new_pollution.xlsx')